In [0]:
%pip install chromadb --quiet

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
databricks-feature-engineering 0.2.0 requires pyspark<4,>=3.1.2, which is not installed.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


#Evaluation of Retrieval System with MLflow
The question: "How well does the retriever work with a given query?"

##Creating Ground Truth dataset

The question column contains all the questions that will be evaluated and the source column is the expected source for the answer for the questions as an ordered list of strings.

In [0]:
import os
import pandas as pd
import mlflow
import chromadb
from typing import List
from langchain.chains import RetrievalQA
from langchain.document_loaders import WebBaseLoader
from langchain.llms import Databricks
from langchain.embeddings.databricks import DatabricksEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from chromadb.errors import InvalidDimensionException

In [0]:
eval_data = pd.DataFrame(
    {
        "question": [
            "What is MLflow?",
            "What is Databricks?",
            "How to serve a model on Databricks?",
            "How to enable MLflow Autologging for my workspace by default?",
        ],
        "source": [
            ["https://mlflow.org/docs/latest/index.html"],
            ["https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html"],
            ["https://mlflow.org/docs/latest/python_api/mlflow.deployments.html"],
            ["https://mlflow.org/docs/latest/tracking/autolog.html"],
        ],
    }
)


In [0]:
display(eval_data)

question,source
What is MLflow?,List(https://mlflow.org/docs/latest/index.html)
What is Databricks?,List(https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html)
How to serve a model on Databricks?,List(https://mlflow.org/docs/latest/python_api/mlflow.deployments.html)
How to enable MLflow Autologging for my workspace by default?,List(https://mlflow.org/docs/latest/tracking/autolog.html)


##Evaluate the Embedding Model with MLflow

There are three built-in metrics mlflow.metrics.precision_at_k(k),  mlflow.metrics.recall_at_k(k) and mlflow.metrics.ndcg_at_k(k) to help determine how effective your retriever is at predicting the most relevant results for you. For example, suppose the vector database returns 10 results (k=10), and out of these 10 results, 4 are relevant to your query. The precision_at_10 would be 4/10 or 40%. 

In [0]:
loader = WebBaseLoader(
    [ 
     "https://mlflow.org/docs/latest/index.html",
     "https://mlflow.org/docs/latest/tracking/autolog.html", 
     "https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html",
     "https://mlflow.org/docs/latest/python_api/mlflow.deployments.html",
     "https://mlflow.org/docs/latest/model-evaluation/index.html",
     "https://mlflow.org/docs/latest/models.html",
     "https://mlflow.org/docs/latest/deep-learning/index.html"])

documents = loader.load()

In [0]:
def evaluate_embedding(embedding_function):
    CHUNK_SIZE = 1000
    list_of_documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=0, separator="\n")
    docs = text_splitter.split_documents(list_of_documents)
    try:
        retriever = Chroma.from_documents(docs, embedding_function).as_retriever()
    except InvalidDimensionException:
        Chroma().delete_collection()
        retriever = Chroma.from_documents(docs, embedding_function).as_retriever()

    def retrieve_doc_ids(question: str) -> List[str]:
        docs = retriever.get_relevant_documents(question)
        doc_ids = [doc.metadata["source"] for doc in docs]
        return doc_ids

    def retriever_model_function(question_df: pd.DataFrame) -> pd.Series:
        return question_df["question"].apply(retrieve_doc_ids)

    # Evaluating a function
    with mlflow.start_run() as run:
        evaluate_results = mlflow.evaluate(
                model=retriever_model_function,
                data=eval_data,
                model_type="retriever",
                targets="source",
                evaluators="default",
            )
    return evaluate_results

# Make sure you have access to Foundation Model API pay per token
result1 = evaluate_embedding(DatabricksEmbeddings(endpoint="databricks-bge-large-en")) 

# Make sure you have adeployed bge_base_en_v1_5 and bge_small_en_v1_5 behind model serving endpoint
result2 = evaluate_embedding(DatabricksEmbeddings(endpoint="bge_base_en_v1_5"))
result3 = evaluate_embedding(DatabricksEmbeddings(endpoint="bge_small_en_v1_5"))

# Make sure you extrenal model endpoint cread in model serving
result4 = evaluate_embedding(DatabricksEmbeddings(endpoint="timo_lackmann_oai_embedding"))

eval_results_of_retriever_df_bge_large = result1.tables["eval_results_table"]
eval_results_of_retriever_df_bge_base = result2.tables["eval_results_table"]
eval_results_of_retriever_df_bge_small = result3.tables["eval_results_table"]
eval_results_of_retriever_df_openai_ada = result3.tables["eval_results_table"]

display(eval_results_of_retriever_df_bge_large)
display(eval_results_of_retriever_df_bge_base)
display(eval_results_of_retriever_df_bge_small)
display(eval_results_of_retriever_df_openai_ada)

2024/03/15 09:41:53 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2024/03/15 09:41:53 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/03/15 09:41:54 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/03/15 09:41:54 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: precision_at_3
2024/03/15 09:41:54 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: recall_at_3
2024/03/15 09:41:54 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: ndcg_at_3
2024/03/15 09:42:10 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2024/03/15 09:42:10 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/03/15 09:42:11 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/03/15 09:42:11 INFO mlflow.models.evaluation.default_evalua

question,source,outputs,precision_at_3/score,recall_at_3/score,ndcg_at_3/score
What is MLflow?,List(https://mlflow.org/docs/latest/index.html),"List(https://mlflow.org/docs/latest/index.html, https://mlflow.org/docs/latest/index.html, https://mlflow.org/docs/latest/deep-learning/index.html, https://mlflow.org/docs/latest/deep-learning/index.html)",0.6666666667000001,1,1.0
What is Databricks?,List(https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html),"List(https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html)",1.0,1,1.0
How to serve a model on Databricks?,List(https://mlflow.org/docs/latest/python_api/mlflow.deployments.html),"List(https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html)",0.33333333330000003,1,0.6934264036000001
How to enable MLflow Autologging for my workspace by default?,List(https://mlflow.org/docs/latest/tracking/autolog.html),"List(https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html)",1.0,1,1.0


question,source,outputs,precision_at_3/score,recall_at_3/score,ndcg_at_3/score
What is MLflow?,List(https://mlflow.org/docs/latest/index.html),"List(https://mlflow.org/docs/latest/index.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html, https://mlflow.org/docs/latest/index.html, https://mlflow.org/docs/latest/deep-learning/index.html)",0.6666666667000001,1,0.9197207891
What is Databricks?,List(https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html),"List(https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html)",0.6666666667000001,1,1.0
How to serve a model on Databricks?,List(https://mlflow.org/docs/latest/python_api/mlflow.deployments.html),"List(https://mlflow.org/docs/latest/models.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html)",0.0,0,0.30657359640000004
How to enable MLflow Autologging for my workspace by default?,List(https://mlflow.org/docs/latest/tracking/autolog.html),"List(https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html)",1.0,1,1.0


question,source,outputs,precision_at_3/score,recall_at_3/score,ndcg_at_3/score
What is MLflow?,List(https://mlflow.org/docs/latest/index.html),"List(https://mlflow.org/docs/latest/index.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html, https://mlflow.org/docs/latest/deep-learning/index.html, https://mlflow.org/docs/latest/index.html)",0.33333333330000003,1,1.0
What is Databricks?,List(https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html),"List(https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html)",0.6666666667000001,1,0.9197207891
How to serve a model on Databricks?,List(https://mlflow.org/docs/latest/python_api/mlflow.deployments.html),"List(https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html)",0.6666666667000001,1,0.6934264036000001
How to enable MLflow Autologging for my workspace by default?,List(https://mlflow.org/docs/latest/tracking/autolog.html),"List(https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html)",1.0,1,1.0


question,source,outputs,precision_at_3/score,recall_at_3/score,ndcg_at_3/score
What is MLflow?,List(https://mlflow.org/docs/latest/index.html),"List(https://mlflow.org/docs/latest/index.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html, https://mlflow.org/docs/latest/deep-learning/index.html, https://mlflow.org/docs/latest/index.html)",0.33333333330000003,1,1.0
What is Databricks?,List(https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html),"List(https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html)",0.6666666667000001,1,0.9197207891
How to serve a model on Databricks?,List(https://mlflow.org/docs/latest/python_api/mlflow.deployments.html),"List(https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html)",0.6666666667000001,1,0.6934264036000001
How to enable MLflow Autologging for my workspace by default?,List(https://mlflow.org/docs/latest/tracking/autolog.html),"List(https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html)",1.0,1,1.0


##Evaluate retriever with different Top K values with MLflow

In [0]:
# Evaluating a static evaluation dataset
with mlflow.start_run() as run:
    evaluate_results = mlflow.evaluate(
    data=eval_results_of_retriever_df_bge1,
    targets="source",
    predictions="outputs",
    evaluators="default",
    extra_metrics=[
        mlflow.metrics.precision_at_k(1),
        mlflow.metrics.precision_at_k(2),
        mlflow.metrics.precision_at_k(3),
        mlflow.metrics.recall_at_k(1),
        mlflow.metrics.recall_at_k(2),
        mlflow.metrics.recall_at_k(3),
        mlflow.metrics.ndcg_at_k(1),
        mlflow.metrics.ndcg_at_k(2),
        mlflow.metrics.ndcg_at_k(3),
    ],
)

display(evaluate_results.tables["eval_results_table"])

/databricks/python/lib/python3.10/site-packages/mlflow/data/pandas_dataset.py:134: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  return _infer_schema(self._df)
2024/03/15 09:42:57 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2024/03/15 09:42:57 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/03/15

question,precision_at_3/score,recall_at_3/score,ndcg_at_3/score,source,outputs,precision_at_1/score,precision_at_2/score,recall_at_1/score,recall_at_2/score,ndcg_at_1/score,ndcg_at_2/score
What is MLflow?,0.6666666667000001,1,1.0,List(https://mlflow.org/docs/latest/index.html),"List(https://mlflow.org/docs/latest/index.html, https://mlflow.org/docs/latest/index.html, https://mlflow.org/docs/latest/deep-learning/index.html, https://mlflow.org/docs/latest/deep-learning/index.html)",1,1,1,1,1,1.0
What is Databricks?,1.0,1,1.0,List(https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html),"List(https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html)",1,1,1,1,1,1.0
How to serve a model on Databricks?,0.33333333330000003,1,0.6934264036000001,List(https://mlflow.org/docs/latest/python_api/mlflow.deployments.html),"List(https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html)",0,0,0,0,0,0.38685280720000004
How to enable MLflow Autologging for my workspace by default?,1.0,1,1.0,List(https://mlflow.org/docs/latest/tracking/autolog.html),"List(https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html)",1,1,1,1,1,1.0


##Evaluate the Chunking Strategy with MLflow

In [0]:
def evaluate_chunk_size(chunk_size):
  list_of_documents = loader.load()
  text_splitter = CharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=0,separator="\n")
  docs = text_splitter.split_documents(list_of_documents)
  embedding_function = DatabricksEmbeddings(endpoint="databricks-bge-large-en")
  
  try:
      retriever = Chroma.from_documents(docs, embedding_function).as_retriever()
  except InvalidDimensionException:
      Chroma().delete_collection()
      retriever = Chroma.from_documents(docs, embedding_function).as_retriever()
  
  def retrieve_doc_ids(question: str) -> List[str]:
    docs = retriever.get_relevant_documents(question)
    doc_ids = [doc.metadata["source"] for doc in docs]
    return doc_ids
   
  def retriever_model_function(question_df: pd.DataFrame) -> pd.Series:
    return question_df["question"].apply(retrieve_doc_ids)

  with mlflow.start_run() as run:
      evaluate_results = mlflow.evaluate(
          model=retriever_model_function,
          data=eval_data,
          model_type="retriever",
          targets="source",
          evaluators="default",
      )
  return evaluate_results

result1 = evaluate_chunk_size(500)
result2 = evaluate_chunk_size(1000)
result3 = evaluate_chunk_size(2000)

display(result1.tables["eval_results_table"])
display(result2.tables["eval_results_table"])
display(result3.tables["eval_results_table"])

2024/03/15 09:43:26 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2024/03/15 09:43:26 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/03/15 09:43:27 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/03/15 09:43:27 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: precision_at_3
2024/03/15 09:43:27 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: recall_at_3
2024/03/15 09:43:27 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: ndcg_at_3
2024/03/15 09:43:38 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2024/03/15 09:43:38 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/03/15 09:43:39 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/03/15 09:43:39 INFO mlflow.models.evaluation.default_evalua

question,source,outputs,precision_at_3/score,recall_at_3/score,ndcg_at_3/score
What is MLflow?,List(https://mlflow.org/docs/latest/index.html),"List(https://mlflow.org/docs/latest/index.html, https://mlflow.org/docs/latest/index.html, https://mlflow.org/docs/latest/index.html, https://mlflow.org/docs/latest/index.html)",1.0,1,1
What is Databricks?,List(https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html),"List(https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html)",0.6666666667000001,1,1
How to serve a model on Databricks?,List(https://mlflow.org/docs/latest/python_api/mlflow.deployments.html),"List(https://mlflow.org/docs/latest/python_api/mlflow.deployments.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html)",1.0,1,1
How to enable MLflow Autologging for my workspace by default?,List(https://mlflow.org/docs/latest/tracking/autolog.html),"List(https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html)",1.0,1,1


question,source,outputs,precision_at_3/score,recall_at_3/score,ndcg_at_3/score
What is MLflow?,List(https://mlflow.org/docs/latest/index.html),"List(https://mlflow.org/docs/latest/index.html, https://mlflow.org/docs/latest/index.html, https://mlflow.org/docs/latest/index.html, https://mlflow.org/docs/latest/index.html)",1.0,1,1.0
What is Databricks?,List(https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html),"List(https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html)",1.0,1,1.0
How to serve a model on Databricks?,List(https://mlflow.org/docs/latest/python_api/mlflow.deployments.html),"List(https://mlflow.org/docs/latest/models.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html)",0.33333333330000003,1,0.6309297536
How to enable MLflow Autologging for my workspace by default?,List(https://mlflow.org/docs/latest/tracking/autolog.html),"List(https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html)",1.0,1,1.0


question,source,outputs,precision_at_3/score,recall_at_3/score,ndcg_at_3/score
What is MLflow?,List(https://mlflow.org/docs/latest/index.html),"List(https://mlflow.org/docs/latest/index.html, https://mlflow.org/docs/latest/index.html, https://mlflow.org/docs/latest/index.html, https://mlflow.org/docs/latest/index.html)",1.0,1,1.0
What is Databricks?,List(https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html),"List(https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html)",1.0,1,1.0
How to serve a model on Databricks?,List(https://mlflow.org/docs/latest/python_api/mlflow.deployments.html),"List(https://mlflow.org/docs/latest/models.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html)",0.33333333330000003,1,0.6309297536
How to enable MLflow Autologging for my workspace by default?,List(https://mlflow.org/docs/latest/tracking/autolog.html),"List(https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html)",1.0,1,1.0


#Evaluation of GenAI results with MLflow
The question: "How good is the response of the GenAI app with a given prompt and context?"

##Create a GenAI app with LangChain using docs from the MLflow website

In [0]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatDatabricks
from mlflow.deployments import set_deployments_target
from  mlflow.metrics.genai.metric_definitions import relevance

loader = WebBaseLoader(
    [ 
     "https://mlflow.org/docs/latest/index.html",
     "https://mlflow.org/docs/latest/tracking/autolog.html", 
     "https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html",
     "https://mlflow.org/docs/latest/python_api/mlflow.deployments.html",
     "https://mlflow.org/docs/latest/model-evaluation/index.html",
     "https://mlflow.org/docs/latest/models.html",
     "https://mlflow.org/docs/latest/deep-learning/index.html"])

documents = loader.load()
CHUNK_SIZE = 1000
text_splitter = CharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=0, separator="\n")
texts = text_splitter.split_documents(documents)

def evaluate_model(model_name, eval_df):

    llm = ChatDatabricks(endpoint=model_name, max_tokens=500)

    # create the embedding function using Databricks Foundation Model APIs
    embedding_function = DatabricksEmbeddings(endpoint="databricks-bge-large-en")
    retriever = Chroma.from_documents(texts, embedding_function).as_retriever(fetch_k=3)

    def model(input_df):
        return input_df["questions"].map(qa).tolist()

    qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True
    )

    set_deployments_target("databricks") # To retrieve all endpoint in your Databricks Workspace

    #You can also use any model you have hosted on Databricks, models from the Marketplace or models in the Foundation model API
    relevance_metric = relevance(model=f"endpoints:/databricks-mixtral-8x7b-instruct") 

    with mlflow.start_run():
        results =  mlflow.evaluate(
            model,
            eval_df,
            model_type="question-answering",
            evaluators="default",
            predictions="result",
            extra_metrics=[relevance_metric, mlflow.metrics.latency()],
            evaluator_config={
                "col_mapping": {
                    "inputs": "questions",
                    "context": "source_documents",
                }
            }
        )

    return results

In [0]:
eval_df = pd.DataFrame(
    {
        "questions": [
            "What is MLflow?",
            "What is Databricks?",
            "How to serve a model on Databricks?",
            "How to enable MLflow Autologging for my workspace by default?",
        ],
    }
)

result_llama = evaluate_model("databricks-llama-2-70b-chat", eval_df)
result_mixtral = evaluate_model("databricks-mixtral-8x7b-instruct", eval_df)

2024/03/15 12:11:47 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2024/03/15 12:11:47 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/03/15 12:12:02 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/03/15 12:12:02 WARNING mlflow.metrics.metric_definitions: Failed to load flesch kincaid metric, skipping metric logging.
2024/03/15 12:12:02 WARNING mlflow.metrics.metric_definitions: Failed to load automated readability index metric, skipping metric logging.


  0%|          | 0/1 [00:00<?, ?it/s]

2024/03/15 12:12:04 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: token_count
2024/03/15 12:12:04 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: toxicity
2024/03/15 12:12:06 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: flesch_kincaid_grade_level
2024/03/15 12:12:06 WARNING mlflow.metrics.metric_definitions: Failed to load flesch kincaid metric, skipping metric logging.
2024/03/15 12:12:06 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: ari_grade_level
2024/03/15 12:12:06 WARNING mlflow.metrics.metric_definitions: Failed to load automated readability index metric, skipping metric logging.
2024/03/15 12:12:06 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: exact_match
2024/03/15 12:12:06 INFO mlflow.models.evaluation.default_evaluator: Evaluating metrics: relevance


  0%|          | 0/4 [00:00<?, ?it/s]

2024/03/15 12:12:18 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2024/03/15 12:12:18 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/03/15 12:12:36 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/03/15 12:12:36 WARNING mlflow.metrics.metric_definitions: Failed to load flesch kincaid metric, skipping metric logging.
2024/03/15 12:12:36 WARNING mlflow.metrics.metric_definitions: Failed to load automated readability index metric, skipping metric logging.


  0%|          | 0/1 [00:00<?, ?it/s]

2024/03/15 12:12:38 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: token_count
2024/03/15 12:12:38 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: toxicity
2024/03/15 12:12:41 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: flesch_kincaid_grade_level
2024/03/15 12:12:41 WARNING mlflow.metrics.metric_definitions: Failed to load flesch kincaid metric, skipping metric logging.
2024/03/15 12:12:41 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: ari_grade_level
2024/03/15 12:12:41 WARNING mlflow.metrics.metric_definitions: Failed to load automated readability index metric, skipping metric logging.
2024/03/15 12:12:41 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: exact_match
2024/03/15 12:12:41 INFO mlflow.models.evaluation.default_evaluator: Evaluating metrics: relevance


  0%|          | 0/4 [00:00<?, ?it/s]

In [0]:
display(result_llama.tables["eval_results_table"])

questions outputs source_documents latency token_count toxicity/v1/score relevance/v1/score relevance/v1/justification What is MLflow? MLflow is an open-source platform designed to assist machine learning practitioners and teams in handling the complexities of the machine learning process. It focuses on the full lifecycle of machine learning projects, ensuring that each phase is manageable, traceable, and reproducible. List(List(List(), List(), List(en, https://mlflow.org/docs/latest/index.html, MLflow: A Tool for Managing the Machine Learning Lifecycle — MLflow 2.11.1 documentation), MLflow: A Tool for Managing the Machine Learning Lifecycle — MLflow 2.11.1 documentation
2.11.1
 MLflow
What is MLflow?
Getting Started with MLflow
New Features
LLMs
Model Evaluation
Deep Learning
Traditional ML
Deployment
MLflow Tracking
System Metrics
MLflow Projects
MLflow Models
MLflow Model Registry
MLflow Recipes
MLflow Plugins
MLflow Authentication
Command-Line Interface
Search Runs
Search Experiments
Python API
R API
Java API
REST API
Official MLflow Docker Image
Community Model Flavors
Tutorials and Examples
Contribute
Documentation 
MLflow: A Tool for Managing the Machine Learning Lifecycle
MLflow: A Tool for Managing the Machine Learning Lifecycle 
MLflow is an open-source platform, purpose-built to assist machine learning practitioners and teams in
handling the complexities of the machine learning process. MLflow focuses on the full lifecycle for
machine learning projects, ensuring that each phase is manageable, traceable, and reproducible., Document), List(List(), List(), List(en, https://mlflow.org/docs/latest/index.html, MLflow: A Tool for Managing the Machine Learning Lifecycle — MLflow 2.11.1 documentation), MLflow: A Tool for Managing the Machine Learning Lifecycle — MLflow 2.11.1 documentation
2.11.1
 MLflow
What is MLflow?
Getting Started with MLflow
New Features
LLMs
Model Evaluation
Deep Learning
Traditional ML
Deployment
MLflow Tracking
System Metrics
MLflow Projects
MLflow Models
MLflow Model Registry
MLflow Recipes
MLflow Plugins
MLflow Authentication
Command-Line Interface
Search Runs
Search Experiments
Python API
R API
Java API
REST API
Official MLflow Docker Image
Community Model Flavors
Tutorials and Examples
Contribute
Documentation 
MLflow: A Tool for Managing the Machine Learning Lifecycle
MLflow: A Tool for Managing the Machine Learning Lifecycle 
MLflow is an open-source platform, purpose-built to assist machine learning practitioners and teams in
handling the complexities of the machine learning process. MLflow focuses on the full lifecycle for
machine learning projects, ensuring that each phase is manageable, traceable, and reproducible., Document), List(List(), List(), List(en, https://mlflow.org/docs/latest/index.html, MLflow: A Tool for Managing the Machine Learning Lifecycle — MLflow 2.11.1 documentation), MLflow: A Tool for Managing the Machine Learning Lifecycle — MLflow 2.11.1 documentation
2.11.1
 MLflow
What is MLflow?
Getting Started with MLflow
New Features
LLMs
Model Evaluation
Deep Learning
Traditional ML
Deployment
MLflow Tracking
System Metrics
MLflow Projects
MLflow Models
MLflow Model Registry
MLflow Recipes
MLflow Plugins
MLflow Authentication
Command-Line Interface
Search Runs
Search Experiments
Python API
R API
Java API
REST API
Official MLflow Docker Image
Community Model Flavors
Tutorials and Examples
Contribute
Documentation 
MLflow: A Tool for Managing the Machine Learning Lifecycle
MLflow: A Tool for Managing the Machine Learning Lifecycle 
MLflow is an open-source platform, purpose-built to assist machine learning practitioners and teams in
handling the complexities of the machine learning process. MLflow focuses on the full lifecycle for
machine learning projects, ensuring that each phase is manageable, traceable, and reproducible., Document), List(List(), List(), List(en, https://mlflow.org/docs/latest/index.html, MLflow: A Tool for Managing the Machine Learning Lifecycle — MLflow 2.

In [0]:
display(result_mixtral.tables["eval_results_table"])

questions outputs source_documents latency token_count toxicity/v1/score relevance/v1/score relevance/v1/justification What is MLflow? 
MLflow is an open-source platform designed to help machine learning practitioners and teams manage the complexities of the machine learning process. It focuses on the full lifecycle of machine learning projects, ensuring that each phase is manageable, traceable, and reproducible. List(List(List(), List(), List(en, https://mlflow.org/docs/latest/index.html, MLflow: A Tool for Managing the Machine Learning Lifecycle — MLflow 2.11.1 documentation), MLflow: A Tool for Managing the Machine Learning Lifecycle — MLflow 2.11.1 documentation
2.11.1
 MLflow
What is MLflow?
Getting Started with MLflow
New Features
LLMs
Model Evaluation
Deep Learning
Traditional ML
Deployment
MLflow Tracking
System Metrics
MLflow Projects
MLflow Models
MLflow Model Registry
MLflow Recipes
MLflow Plugins
MLflow Authentication
Command-Line Interface
Search Runs
Search Experiments
Python API
R API
Java API
REST API
Official MLflow Docker Image
Community Model Flavors
Tutorials and Examples
Contribute
Documentation 
MLflow: A Tool for Managing the Machine Learning Lifecycle
MLflow: A Tool for Managing the Machine Learning Lifecycle 
MLflow is an open-source platform, purpose-built to assist machine learning practitioners and teams in
handling the complexities of the machine learning process. MLflow focuses on the full lifecycle for
machine learning projects, ensuring that each phase is manageable, traceable, and reproducible., Document), List(List(), List(), List(en, https://mlflow.org/docs/latest/index.html, MLflow: A Tool for Managing the Machine Learning Lifecycle — MLflow 2.11.1 documentation), MLflow: A Tool for Managing the Machine Learning Lifecycle — MLflow 2.11.1 documentation
2.11.1
 MLflow
What is MLflow?
Getting Started with MLflow
New Features
LLMs
Model Evaluation
Deep Learning
Traditional ML
Deployment
MLflow Tracking
System Metrics
MLflow Projects
MLflow Models
MLflow Model Registry
MLflow Recipes
MLflow Plugins
MLflow Authentication
Command-Line Interface
Search Runs
Search Experiments
Python API
R API
Java API
REST API
Official MLflow Docker Image
Community Model Flavors
Tutorials and Examples
Contribute
Documentation 
MLflow: A Tool for Managing the Machine Learning Lifecycle
MLflow: A Tool for Managing the Machine Learning Lifecycle 
MLflow is an open-source platform, purpose-built to assist machine learning practitioners and teams in
handling the complexities of the machine learning process. MLflow focuses on the full lifecycle for
machine learning projects, ensuring that each phase is manageable, traceable, and reproducible., Document), List(List(), List(), List(en, https://mlflow.org/docs/latest/index.html, MLflow: A Tool for Managing the Machine Learning Lifecycle — MLflow 2.11.1 documentation), MLflow: A Tool for Managing the Machine Learning Lifecycle — MLflow 2.11.1 documentation
2.11.1
 MLflow
What is MLflow?
Getting Started with MLflow
New Features
LLMs
Model Evaluation
Deep Learning
Traditional ML
Deployment
MLflow Tracking
System Metrics
MLflow Projects
MLflow Models
MLflow Model Registry
MLflow Recipes
MLflow Plugins
MLflow Authentication
Command-Line Interface
Search Runs
Search Experiments
Python API
R API
Java API
REST API
Official MLflow Docker Image
Community Model Flavors
Tutorials and Examples
Contribute
Documentation 
MLflow: A Tool for Managing the Machine Learning Lifecycle
MLflow: A Tool for Managing the Machine Learning Lifecycle 
MLflow is an open-source platform, purpose-built to assist machine learning practitioners and teams in
handling the complexities of the machine learning process. MLflow focuses on the full lifecycle for
machine learning projects, ensuring that each phase is manageable, traceable, and reproducible., Document), List(List(), List(), List(en, https://mlflow.org/docs/latest/index.html, MLflow: A Tool for Managing the Machine Learning Lifecycle — MLflow 2.11.1 d